## Описание проекта

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Описание данных

**Признаки:**
- *DateCrawled* — дата скачивания анкеты из базы
- *VehicleType* — тип автомобильного кузова
- *RegistrationYear* — год регистрации автомобиля
- *Gearbox* — тип коробки передач
- *Power* — мощность (л. с.)
- *Model* — модель автомобиля
- *Kilometer* — пробег (км)
- *RegistrationMonth* — месяц регистрации автомобиля
- *FuelType* — тип топлива
- *Brand* — марка автомобиля
- *NotRepaired* — была машина в ремонте или нет
- *DateCreated* — дата создания анкеты
- *NumberOfPictures* — количество фотографий автомобиля
- *PostalCode* — почтовый индекс владельца анкеты (пользователя)
- *LastSeen* — дата последней активности пользователя

**Целевой признак:**
- *Price* — цена (евро)

## 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

from IPython.display import display

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression

from catboost import Pool
from catboost import CatBoostRegressor

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn import preprocessing

Определим константные значения, которые будем использовать в дальнейшем.

In [2]:
# Параметр рандомизиции
RAND_SEED = 12345

Прочитаем данные из файла и выведем первые 5 строк.

In [3]:
autos = pd.read_csv('/datasets/autos.csv')
autos.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Посмотрим подробнее на названия столбцов.

In [4]:
autos.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

Видно, что названия столбцов состоят только из латинских символов и не содержат лишних пробелов в начале и в конце. Однако, они содержат заглавные буквы. Переведем названия в нижний регистр, а чтобы отделить слова, поставим между ними нижнее подчеркивание. Таким обзом, в дальнейшем мы уменьшим вероятность опечаток.

In [5]:
# получаем новые названия, проставляя знак '_' перед каждой неначальной заглавной буквой
new_columns = list(map(lambda x: re.sub(r'(\B[A-Z])', r'_\1', x), autos.columns.values))

In [6]:
# приведем все буквы к нижнему регистру
new_columns = list(map(str.lower, new_columns))

In [7]:
# поменяем названия в исходной таблице
autos.columns = new_columns
autos.columns

Index(['date_crawled', 'price', 'vehicle_type', 'registration_year', 'gearbox',
       'power', 'model', 'kilometer', 'registration_month', 'fuel_type',
       'brand', 'not_repaired', 'date_created', 'number_of_pictures',
       'postal_code', 'last_seen'],
      dtype='object')

Проверим, есть ли пустые значения в таблице.

In [8]:
autos.isnull().sum()

date_crawled              0
price                     0
vehicle_type          37490
registration_year         0
gearbox               19833
power                     0
model                 19705
kilometer                 0
registration_month        0
fuel_type             32895
brand                     0
not_repaired          71154
date_created              0
number_of_pictures        0
postal_code               0
last_seen                 0
dtype: int64

Пропущенные значения есть. Посмотрим внимательнее на данные столбцы и решим, каким образом избавиться от пропусков.

Первый столбец - это `vehicle_type` (тип автомобильного кузова).

In [9]:
autos['vehicle_type'].value_counts(dropna=False)

sedan          91457
small          79831
wagon          65166
NaN            37490
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: vehicle_type, dtype: int64

In [10]:
autos.loc[autos['vehicle_type'].isnull()].head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
16,2016-04-01 12:46:46,300,NaN,2016,NaN,60,polo,150000,0,petrol,volkswagen,NaN,2016-04-01 00:00:00,0,38871,2016-04-01 12:46:46
22,2016-03-23 14:52:51,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no,2016-03-23 00:00:00,0,49716,2016-03-31 01:16:33
26,2016-03-10 19:38:18,5555,NaN,2017,manual,125,c4,125000,4,NaN,citroen,no,2016-03-10 00:00:00,0,31139,2016-03-16 09:16:46
31,2016-03-29 16:57:02,899,NaN,2016,manual,60,clio,150000,6,petrol,renault,NaN,2016-03-29 00:00:00,0,37075,2016-03-29 17:43:07


Из таблицы видно, что пропуски могут быть для различных моделей различных марок. Также видно, что это категориальная переменная. Клиент не ввел данную информацию, значит или он не смог определить какой тип кузова, или это не имеет для него значения. У нас есть категория *other*. Под ней можно понимать что угодно, а также неопределенные значения. Так что заполним пропуски в этом столбце этим значением.

In [11]:
autos['vehicle_type'] = autos['vehicle_type'].fillna('other')
autos['vehicle_type'].value_counts(dropna=False)

sedan          91457
small          79831
wagon          65166
other          40778
bus            28775
convertible    20203
coupe          16163
suv            11996
Name: vehicle_type, dtype: int64

Сразу же посмотрим на столбцы `model` (модель автомобиля), `fuel_type` (тип топлива).

In [12]:
print('model')
display(autos['model'].value_counts(dropna=False))

print('-' * 60)

print('fuel_type')
display(autos['fuel_type'].value_counts(dropna=False))

model


golf                  29232
other                 24421
3er                   19761
NaN                   19705
polo                  13066
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
serie_1                   2
range_rover_evoque        2
Name: model, Length: 251, dtype: int64

------------------------------------------------------------
fuel_type


petrol      216352
gasoline     98720
NaN          32895
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: fuel_type, dtype: int64

Видно, причина здесь та же самая, что и для столбца `vehicle_type`. Таким образом, проведем аналогичную замену на значение *other*.

In [13]:
autos['model'] = autos['model'].fillna('other')
print('model')
display(autos['model'].value_counts(dropna=False))

print('-' * 60)

autos['fuel_type'] = autos['fuel_type'].fillna('other')
print('fuel_type')
display(autos['fuel_type'].value_counts(dropna=False))

model


other                 44126
golf                  29232
3er                   19761
polo                  13066
corsa                 12570
                      ...  
serie_2                   8
serie_3                   4
rangerover                4
range_rover_evoque        2
serie_1                   2
Name: model, Length: 250, dtype: int64

------------------------------------------------------------
fuel_type


petrol      216352
gasoline     98720
other        33099
lpg           5310
cng            565
hybrid         233
electric        90
Name: fuel_type, dtype: int64

Далее посмотрим на столбец `gearbox` (тип коробки передач).

In [14]:
autos['gearbox'].value_counts(dropna=False)

manual    268251
auto       66285
NaN        19833
Name: gearbox, dtype: int64

В данном случае есть всего два значения - механическая и автоматическая. Мы не можем однозначно определить какой тип будет у машин с пропуском в данном столбце. Удалять также не стоит, значений достаточно много. Поэтому введем еще одно дополнительное значение *no_info*. Значение *other* не стоит использовать, так как это может запутать того, кто будет смотерть в измененную таблицу.

In [15]:
autos['gearbox'] = autos['gearbox'].fillna('no_info')

Последний столбец с пропусками - `not_repaired` (была машина в ремонте или нет).

In [16]:
display(autos['not_repaired'].value_counts(dropna=False))

no     247161
NaN     71154
yes     36054
Name: not_repaired, dtype: int64

Аналогичная ситуация, как со столбцом `gearbox`. Заполним значением *no_info*.

In [17]:
autos['not_repaired'] = autos['not_repaired'].fillna('no_info')

Проверим, что мы действильно заполнили все пропуски.

In [18]:
autos.isnull().sum()

date_crawled          0
price                 0
vehicle_type          0
registration_year     0
gearbox               0
power                 0
model                 0
kilometer             0
registration_month    0
fuel_type             0
brand                 0
not_repaired          0
date_created          0
number_of_pictures    0
postal_code           0
last_seen             0
dtype: int64

Теперь посмотрим на общую информацию по таблице.

In [19]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
date_crawled          354369 non-null object
price                 354369 non-null int64
vehicle_type          354369 non-null object
registration_year     354369 non-null int64
gearbox               354369 non-null object
power                 354369 non-null int64
model                 354369 non-null object
kilometer             354369 non-null int64
registration_month    354369 non-null int64
fuel_type             354369 non-null object
brand                 354369 non-null object
not_repaired          354369 non-null object
date_created          354369 non-null object
number_of_pictures    354369 non-null int64
postal_code           354369 non-null int64
last_seen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


Теперь проверим правильные ли типы данных у столбцов.<br>
`date_crawled`, `date_created`, `last_seen` - это даты, а значит тип данных datatime<br>
Остальные типы данных верные.

In [20]:
autos.head()

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,not_repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,other,1993,manual,0,golf,150000,0,petrol,volkswagen,no_info,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,other,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,no_info,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [21]:
autos['date_crawled'] = pd.to_datetime(autos['date_crawled'], format='%Y-%m-%dT%H:%M:%S')
autos['date_created'] = pd.to_datetime(autos['date_created'], format='%Y-%m-%dT%H:%M:%S')
autos['last_seen'] = pd.to_datetime(autos['last_seen'], format='%Y-%m-%dT%H:%M:%S')

In [22]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
date_crawled          354369 non-null datetime64[ns]
price                 354369 non-null int64
vehicle_type          354369 non-null object
registration_year     354369 non-null int64
gearbox               354369 non-null object
power                 354369 non-null int64
model                 354369 non-null object
kilometer             354369 non-null int64
registration_month    354369 non-null int64
fuel_type             354369 non-null object
brand                 354369 non-null object
not_repaired          354369 non-null object
date_created          354369 non-null datetime64[ns]
number_of_pictures    354369 non-null int64
postal_code           354369 non-null int64
last_seen             354369 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(7), object(6)
memory usage: 43.3+ MB


Мы преподготовили данные для анализа. Теперь посмотрим на адекватность значений и логические ограничения .

In [23]:
autos_modified = autos.copy()

Посмотрим на столбец `registration_year` (год регистрации автомобиля)

In [24]:
autos_modified['registration_year'].value_counts()

2000    24490
1999    22728
2005    22109
2001    20124
2006    19900
        ...  
3200        1
1920        1
1919        1
1915        1
8455        1
Name: registration_year, Length: 151, dtype: int64

Видно, что присутствуют года, совершенно нереальные (например, 3200 и 8455). Посмотрим, сколько таких годов, и удалим такие строки.

In [25]:
autos_modified.query('(registration_year < 1900) or (registration_year > 2021)')['registration_year'].value_counts()

1000    37
9999    26
5000    17
3000     7
1800     5
1500     5
6000     5
1234     4
2500     4
7000     4
1111     3
9000     3
4000     3
2800     2
4500     2
8000     2
2222     2
5555     2
5911     2
1300     2
1600     2
8500     1
2066     1
1602     1
9229     1
8200     1
5900     1
7500     1
2900     1
8455     1
4100     1
2290     1
7800     1
6500     1
3700     1
9450     1
1001     1
1255     1
1253     1
5600     1
3800     1
4800     1
7100     1
8888     1
5300     1
1200     1
3500     1
1688     1
2200     1
9996     1
3200     1
1400     1
1039     1
Name: registration_year, dtype: int64

In [26]:
index_for_drop = autos_modified.query('(registration_year < 1900) or (registration_year > 2021)').index
autos_modified = autos_modified.drop(index_for_drop).reset_index(drop=True)
print('Rows with correct years: ', len(autos_modified.query('(registration_year >= 1900) and (registration_year <= 2021)')))
print('All rows: ', autos_modified.shape[0])

Rows with correct years:  354198
All rows:  354198


Посмотрим на столбец `registration_month` (месяц регистрации автомобиля).

In [27]:
autos_modified['registration_month'].value_counts()

0     37220
3     34368
6     31501
4     29266
5     29149
7     27210
10    26098
12    24287
11    24184
9     23811
1     23214
8     22626
2     21264
Name: registration_month, dtype: int64

Видно, что существует много строк с месяцем равным 0. Это неверные значения (месяца с таким номером нет), но как-то изменять их нельзя. Скорее всего это были пропуски, и автоматиески они заполнились нулями. Однозначно сейчас мы сказать это не можем. Однако в дальнейшем предсказывать по таким данным некорректно, так как таких значений достаточно много, и они будут влиять не предсказания. Удалим такие значения. 

In [28]:
index_for_drop = autos_modified.query('registration_month == 0').index
autos_modified = autos_modified.drop(index_for_drop).reset_index(drop=True)
print('Rows with correct month: ', len(autos_modified.query('registration_year != 0')))
print('All rows: ', autos_modified.shape[0])

Rows with correct month:  316978
All rows:  316978


Посмотрим на адекватность значений столбца `power` (мощность (л. с.)).

In [29]:
autos_modified['power'].value_counts()

0        24894
75       21537
60       14270
150      13413
140      12342
         ...  
1401         1
6011         1
10110        1
17011        1
7529         1
Name: power, Length: 677, dtype: int64

Видим, что есть нереалистичные значения (например, 17011) - таких машин не бывает. Обычно в автомобиле, не более 2500 л.с. и не менее 1 л.с. Посмотрим, сколько таких значений и удалим.

In [30]:
autos_modified.query('(power > 2500) or (power < 1)')['power'].value_counts()

0        24894
4700         2
7511         2
3500         2
12512        2
         ...  
10110        1
17011        1
10910        1
15020        1
7515         1
Name: power, Length: 79, dtype: int64

In [31]:
index_for_drop = autos_modified.query('(power > 2500) or (power < 1)').index
autos_modified = autos_modified.drop(index_for_drop).reset_index(drop=True)
print('Rows with correct power: ', len(autos_modified.query('(power <= 2500) and (power >= 1)')))
print('All rows: ', autos_modified.shape[0])

Rows with correct power:  292002
All rows:  292002


Также стоит посмотреть на наш целевой признак `price` (цена).

In [32]:
autos_modified['price'].value_counts()

0        4980
1500     4165
500      3977
2500     3526
1200     3506
         ... 
11149       1
7563        1
3977        1
4747        1
4994        1
Name: price, Length: 3574, dtype: int64

Можно заметить, что для 4980 строк, значения целевого признака равно 0. Это некорректно, значит цена не была объявлена. Удалим такие строки.

In [33]:
index_for_drop = autos_modified.query('price == 0').index
autos_modified = autos_modified.drop(index_for_drop).reset_index(drop=True)
print('Rows with correct price: ', len(autos_modified.query('price != 0')))
print('All rows: ', autos_modified.shape[0])

Rows with correct price:  287022
All rows:  287022


Теперь подумаем, какие данные будут важны для определения стоимости автомобиля. Признаки `date_crawled` (дата скачивания анкеты из базы), `date_created` (дата создания анкеты), `postal_code` (почтовый индекс владельца анкеты), `last_seen` (дата последней активности пользователя) не важны для автомобиля, это характеристики пользователя. Поэтому удалим эти колонки.

In [34]:
autos_modified = autos_modified.drop(['date_crawled', 'date_created', 'postal_code', 'last_seen'], axis=1)

In [35]:
autos_modified.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287022 entries, 0 to 287021
Data columns (total 12 columns):
price                 287022 non-null int64
vehicle_type          287022 non-null object
registration_year     287022 non-null int64
gearbox               287022 non-null object
power                 287022 non-null int64
model                 287022 non-null object
kilometer             287022 non-null int64
registration_month    287022 non-null int64
fuel_type             287022 non-null object
brand                 287022 non-null object
not_repaired          287022 non-null object
number_of_pictures    287022 non-null int64
dtypes: int64(6), object(6)
memory usage: 26.3+ MB


Посмотрим на колонкe `number_of_pictures` (количество фотографий автомобиля).

In [36]:
autos_modified['number_of_pictures'].value_counts()

0    287022
Name: number_of_pictures, dtype: int64

Видно, что для всех строк здесь хранится только одно значение 0. А значит при предсказании оно не будет влиять ни на что. Также удалим данный столбец.

In [37]:
autos_modified = autos_modified.drop('number_of_pictures', axis=1)

В итоге у нас получилась такая таблица.

In [38]:
autos_modified.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287022 entries, 0 to 287021
Data columns (total 11 columns):
price                 287022 non-null int64
vehicle_type          287022 non-null object
registration_year     287022 non-null int64
gearbox               287022 non-null object
power                 287022 non-null int64
model                 287022 non-null object
kilometer             287022 non-null int64
registration_month    287022 non-null int64
fuel_type             287022 non-null object
brand                 287022 non-null object
not_repaired          287022 non-null object
dtypes: int64(5), object(6)
memory usage: 24.1+ MB


На данном этапе также выделим отдельно признаки и целевой признак, которые мы будем использовать для обучения и предсказания.

In [39]:
features = autos_modified.drop('price', axis=1)
target = autos_modified['price']

### Вывод

На данном этапе мы загрузили и подготовили данные. Поменяли названия колонок, чтобы использовалиь только прописные буквы и разделили слова нижним подчеркванием. Затем определили строки с пропусками и заполнили их подходящим значением. После скорректировали типы данных. Данные проверили на адекватность и оставили только корректные значения. Также определили какие признаки влияют на  определение стоимости и убрали остальные. В итоге выделили отдельно признаки и целевой признак, чтобы можно было переходить к обучению моделей.

## 2. Обучение моделей

Выделим отдельно количественные и категориальные признаки, которые понадобятся нам в дальнейшем. В данном случае количесвтенными являются `power` и `kilometer`. Признаки `registration year` и `registration_month` являются по сути категориальными, значения обоих столбцов конечное количество категорий (после предварительной фильтрации неадекватных значений).

In [40]:
num_features = ['power', 'kilometer']

In [41]:
cat_features = ['vehicle_type', 'registration_year', 'gearbox', 'model',
                'registration_month', 'fuel_type', 'brand', 'not_repaired']

В дальнейшем мы рассмотрим три разные модели для определения стоимости автомобиля. Поскольку разные модели по разному работают с категориальными переменными, каждый раз мы будем подготавливать признаки заново. Для этого мы будем всегда использовать один и тот же параметр рандомизации и делить их в одинаковом соотношении. Конечно, можно по умолчанию преобразовать все признаки методом прямого отображения, но нам интересно посмотреть на особенности разных моделей. Также каждый раз будем проводить стандартизацию количественных признакнов после разбиения, чтобы избежать утечки данных.

### Линейная регрессия

Линейная регрессия не умеет работать с категориальными признаками. Поэтому необходимо их преобразовать в количественные. Воспользуемся для этого методом прямого отображения (One-Hot Encoding).

In [42]:
features_lr = pd.get_dummies(features, drop_first=True)

Разобьем наши признаки на обучающую, валидационную и тестовые выборки.

In [43]:
# отделим 40% данных для проверочных выборок
features_train_lr, features_check_lr, target_train_lr, target_check_lr = train_test_split(
        features_lr, target, test_size=0.4, random_state=RAND_SEED)

In [44]:
# определим 50% данных для валидационной выборки и 50% для тестовой выборки (из проверочное выборки)
features_valid_lr, features_test_lr, target_valid_lr, target_test_lr = train_test_split(
        features_check_lr, target_check_lr, test_size=0.5, random_state=RAND_SEED)

Проведем стандартизацию количественных признаков.

In [45]:
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train_lr[num_features])

features_train_lr[num_features] = scaler.transform(features_train_lr[num_features])
features_valid_lr[num_features] = scaler.transform(features_valid_lr[num_features])
features_test_lr[num_features] = scaler.transform(features_test_lr[num_features])

Соединим обучающую и валидационные выборки. Обучим на них модель. Данный шаг с разделением, а затем объединением выборок, был сделан по той причине, что в дальнейшем мы будем использовать и обучающую, и валидационную выборки, и уже на них строить окончательную модель и оценивать работу на тестовой выборке. Чтобы не было несоответсвий, мы проделали эту процедуру.

In [46]:
features_train_valid_lr = pd.concat([features_train_lr] + [features_valid_lr])
target_train_valid_lr = pd.concat([target_train_lr] + [target_valid_lr])

In [47]:
model_lr = LinearRegression()

In [48]:
%%time
model_lr.fit(features_train_valid_lr, target_train_valid_lr)

CPU times: user 19.3 s, sys: 4.95 s, total: 24.3 s
Wall time: 24.3 s


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [49]:
%%time
predictions_lr = model_lr.predict(features_test_lr)

CPU times: user 136 ms, sys: 111 ms, total: 247 ms
Wall time: 278 ms


In [50]:
# посчитаем значение RMSE (квадратный корень из средней квадратичной ошибки) через вычисление MSE
mse_lr = mean_squared_error(target_test_lr, predictions_lr)
rmse_lr = mse_lr ** 0.5
print('rmse_lr =', rmse_lr)

rmse_lr = 2738.768703235659


### CatBoost

Разделим наши выборки и проведем стандартизацию. CatBoost умеет работать с категориальными признаками, проверим насколько успешно.

In [51]:
# отделим 40% данных для проверочных выборок
features_train_cbr, features_check_cbr, target_train_cbr, target_check_cbr = train_test_split(
        features, target, test_size=0.4, random_state=RAND_SEED)

In [52]:
# определим 50% данных для валидационной выборки и 50% для тестовой выборки (из проверочное выборки)
features_valid_cbr, features_test_cbr, target_valid_cbr, target_test_cbr = train_test_split(
    features_check_cbr, target_check_cbr, test_size=0.5, random_state=RAND_SEED)

In [53]:
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train_cbr[num_features])

features_train_cbr[num_features] = scaler.transform(features_train_cbr[num_features])
features_valid_cbr[num_features] = scaler.transform(features_valid_cbr[num_features])
features_test_cbr[num_features] = scaler.transform(features_test_cbr[num_features])

С помощью поиска по сетке подберем наилучшие параметры, чтобы метрика RMSE  была минимальна.

In [54]:
model_cbr = CatBoostRegressor(loss_function='RMSE')

Выберем значение перебираемых гиперпараметров. Возьмем лишь небольшое количество значений и введем ограничение на глубину деревьев. Этим мы скоратим время подбора лучших параметров.

In [55]:
grid_cbr = {'iterations': [1000, 2000],
            'learning_rate': [0.9, 1],
            'depth': [1]}

Обучать модель будем на обучающей выборке. По умолчанию, в методе grid_search параметр search_by_train_test_split=True. Это означает что каждый раз валидационная выборка в ходе этого обучения будет браться из обучающих данных.

In [56]:
train_pool = Pool(data=features_train_cbr, label=target_train_cbr, cat_features=cat_features)

In [57]:
result = model_cbr.grid_search(param_grid=grid_cbr, X=train_pool)

0:	loss: 2174.3941759	best: 2174.3941759 (0)	total: 1m 26s	remaining: 4m 18s
1:	loss: 2171.2726648	best: 2171.2726648 (1)	total: 2m 50s	remaining: 2m 50s
2:	loss: 2165.0225041	best: 2165.0225041 (2)	total: 5m 39s	remaining: 1m 53s
3:	loss: 2162.8222367	best: 2162.8222367 (3)	total: 8m 27s	remaining: 0us
Estimating final quality...


Посмотрим какие параметры подобрались для наилучшей модели, при использовани метрики RMSE.

In [58]:
model_cbr.get_params()

{'loss_function': 'RMSE', 'depth': 1, 'learning_rate': 1, 'iterations': 2000}

In [59]:
valid_pool = Pool(data=features_valid_cbr, label=target_valid_cbr, cat_features=cat_features)

Мы уже можем пользоваться нашей моделью. Она уже обучена на оптимальных гиперпараметрах после grid_search. Однако, мы хотим замерить время обучения, с использованием валидационной выборки.

In [60]:
%%time
model_cbr.fit(train_pool, eval_set=valid_pool, verbose=100)

0:	learn: 3566.3372338	test: 3556.0649942	best: 3556.0649942 (0)	total: 169ms	remaining: 5m 36s
100:	learn: 2255.0065650	test: 2268.4568743	best: 2267.0683155 (99)	total: 10.4s	remaining: 3m 15s
200:	learn: 2218.1410366	test: 2228.0906097	best: 2227.9342843 (199)	total: 20.3s	remaining: 3m 1s
300:	learn: 2202.2199994	test: 2210.9783251	best: 2210.9783251 (300)	total: 30.2s	remaining: 2m 50s
400:	learn: 2192.3349483	test: 2202.7247741	best: 2202.7247741 (400)	total: 40.2s	remaining: 2m 40s
500:	learn: 2186.6134889	test: 2195.5223225	best: 2195.5223225 (500)	total: 50.1s	remaining: 2m 29s
600:	learn: 2181.4686581	test: 2190.3377484	best: 2190.3377484 (600)	total: 1m	remaining: 2m 19s
700:	learn: 2177.4256670	test: 2186.3283179	best: 2186.3283179 (700)	total: 1m 9s	remaining: 2m 9s
800:	learn: 2174.6020975	test: 2183.5871124	best: 2183.5151267 (793)	total: 1m 19s	remaining: 1m 59s
900:	learn: 2172.4375388	test: 2181.3776565	best: 2181.3193832 (876)	total: 1m 29s	remaining: 1m 49s
1000:	le

После обучения, мы получили наилучшее значение метрики RMSE на валидационной выборке: 2166.254744

Посмотрим на качество предсказания на тестовой выборке.

In [61]:
%%time
predictions_cbr = model_cbr.predict(features_test_cbr)

CPU times: user 363 ms, sys: 13.4 ms, total: 376 ms
Wall time: 366 ms


In [62]:
# посчитаем значение RMSE (квадратный корень из средней квадратичной ошибки) через вычисление MSE
mse_cbr = mean_squared_error(target_test_cbr, predictions_cbr)
rmse_cbr = mse_cbr ** 0.5
print('rmse_cbr =', rmse_cbr)

rmse_cbr = 2171.0763838064086


### LightGBM

LightGBM напрямую не умеет работать с категориальными переменными. Их необхоимо представить в виде чисел. Используем для этого LabelEncoder.

In [63]:
lbl = preprocessing.LabelEncoder()

In [64]:
features_lgbmr = features.copy()
features_lgbmr[cat_features] = features_lgbmr[cat_features].apply(lbl.fit_transform)

Разделим на выборки и проведем стандартизацию количесвтенных параметров.

In [65]:
# отделим 40% данных для проверочных выборок
features_train_lgbmr, features_check_lgbmr, target_train_lgbmr, target_check_lgbmr = train_test_split(
        features_lgbmr, target, test_size=0.4, random_state=RAND_SEED)

In [66]:
# определим 50% данных для валидационной выборки и 50% для тестовой выборки (из проверочное выборки)
features_valid_lgbmr, features_test_lgbmr, target_valid_lgbmr, target_test_lgbmr = train_test_split(
        features_check_lgbmr, target_check_lgbmr, test_size=0.5, random_state=RAND_SEED)

In [67]:
pd.options.mode.chained_assignment = None

scaler = StandardScaler()
scaler.fit(features_train_lgbmr[num_features])

features_train_lgbmr[num_features] = scaler.transform(features_train_lgbmr[num_features])
features_valid_lgbmr[num_features] = scaler.transform(features_valid_lgbmr[num_features])
features_test_lgbmr[num_features] = scaler.transform(features_test_lgbmr[num_features])

In [68]:
model_lgbmr = lgb.LGBMRegressor(boosting_type='gbdt')

In [69]:
grid_lgbmr = {'n_estimators': [1000, 2000],
              'learning_rate': [0.9, 1],
              'max_depth': [1]}

In [70]:
grid = GridSearchCV(estimator=model_lgbmr, param_grid=grid_lgbmr, verbose=True, cv=3)
grid.fit(features_train_lgbmr, target_train_lgbmr, eval_metric='rmse', categorical_feature=cat_features)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['brand', 'fuel_type', 'gearbox', 'model', 'not_repaired', 'registration_month', 'registration_year', 'vehicle_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['brand', 'fuel_type', 'gearbox', 'model', 'not_repaired', 'registration_month', 'registration_year', 'vehicle_type']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['brand', 'fuel_type', 'gearbox', 'model', 'not_repaired', 'registration_month', '

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=None, random_state=None,
                                     reg_alpha=0.0, reg_lambda=0.0, silent=True,
                                     subsample=1.0, subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.9, 1], 'max_depth': [1],
                         'n_estimators': [1000, 2000]},
             pre_dispatch='2*n_jobs', refit=

In [71]:
print('Best parameters found by grid search are:', grid.best_params_)

Best parameters found by grid search are: {'learning_rate': 1, 'max_depth': 1, 'n_estimators': 2000}


In [72]:
best_lgbmr_learning_rate = grid.best_params_['learning_rate']
best_lgbmr_estimators = grid.best_params_['n_estimators']
best_lgbmr_max_depth = grid.best_params_['max_depth']

In [73]:
best_model_lgbmr = lgb.LGBMRegressor(boosting_type='gbdt', 
                                     learning_rate=best_lgbmr_learning_rate, 
                                     n_estimators=best_lgbmr_estimators,
                                     max_depth=best_lgbmr_max_depth)

In [74]:
%%time
best_model_lgbmr.fit(features_train_lgbmr, target_train_lgbmr, 
                     eval_set=[(features_valid_lgbmr, target_valid_lgbmr)],
                     eval_metric='rmse',
                     categorical_feature=cat_features,
                     verbose=200)

[200]	valid_0's rmse: 2061.4	valid_0's l2: 4.24938e+06
[400]	valid_0's rmse: 2053.11	valid_0's l2: 4.21525e+06
[600]	valid_0's rmse: 2049.64	valid_0's l2: 4.20101e+06
[800]	valid_0's rmse: 2047.04	valid_0's l2: 4.19038e+06
[1000]	valid_0's rmse: 2045.74	valid_0's l2: 4.18504e+06
[1200]	valid_0's rmse: 2044.34	valid_0's l2: 4.17932e+06
[1400]	valid_0's rmse: 2043.7	valid_0's l2: 4.17671e+06
[1600]	valid_0's rmse: 2042.91	valid_0's l2: 4.17347e+06
[1800]	valid_0's rmse: 2042.34	valid_0's l2: 4.17117e+06
[2000]	valid_0's rmse: 2041.73	valid_0's l2: 4.16866e+06
CPU times: user 53.7 s, sys: 0 ns, total: 53.7 s
Wall time: 54.8 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=1, max_depth=1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=2000, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [75]:
%%time
predictions_lgbmr = best_model_lgbmr.predict(features_test_lgbmr)

CPU times: user 4.18 s, sys: 0 ns, total: 4.18 s
Wall time: 4.21 s


In [76]:
# посчитаем значение RMSE (квадратный корень из средней квадратичной ошибки) через вычисление MSE
mse_lgbmr = mean_squared_error(target_test_lgbmr, predictions_lgbmr)
rmse_lgbmr = mse_lgbmr ** 0.5
print('rmse_lgbmr =', rmse_lgbmr)

rmse_lgbmr = 2046.970201558827


### Вывод

На данном шаге изучали модели (Линейная регрессия, CatBoost, LightGBM), подбирали для них оптимальные параметры, чтобы значение RMSE было минимальным. Затем обучали на тренировочной выборке и оценивали качество на валидационной. А затем проверяли насколько хорошо модель предсказывате на тестовых данных.

## 3. Анализ моделей

На предыдущем шаге мы посмотрели на три разные модели. Оценили качество предсказаний на тренировочных выборках.<br>

Значения RMSE для различных моделей:
- Линейная регрессия: &emsp;&nbsp;&nbsp;&nbsp; 2738.768703235659 
- CatBoost: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp; 2171.0763838064086
- LightGBM: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 2046.970201558827

Стоит отметить, что если рассматривать больше параметров для моделей, то результат получится лучше. Однако время поиска параметров также увеличится в разы. Мы ограничились лишь небольшим количеством проверяемых значений.

Также мы измеряли время обучения и время предсказания для каждой модели на оптимальных значениях гиперпараметров.

Время обучения для различных моделей:
- Линейная регрессия: &emsp;&nbsp;&nbsp;&nbsp; 24.3 s
- CatBoost: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp; 3min 22s
- LightGBM: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 54.8 s

Время предсказания для различных моделей:
- Линейная регрессия: &emsp;&nbsp;&nbsp;&nbsp; 278 ms
- CatBoost: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&nbsp; 366 ms
- LightGBM: &emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp; 4.21 s

### Вывод

Мы рассмотрели значение RMSE на тестовых данных. Лучшее значение (самое близкое к 0) получилось для LightGBM. Самое высокое значение этой метрики показала линейная регрессия. Это подтверждает тот факт, что алгоритмы градиентного бустинга с подбором параметров работают качественнее, чем более простая модель без возможности сильно влиять на процесс обучения.<br>
Также мы рассматривали время обучения и предсказания. Самое долгое время обучения показал алгоритм CatBoost. Он дольше LightGBM более чем в 3 раза. LightGBM в свою очередь обучается лишь в 2 раза дольше чем линейная регрессия, с учетом наилучшшего качества. Предсказание дольше всего выдает LightGBM - в 11 раз дольше CatBoost.<br>
Стоит отметить, что и CatBoost, и LightGBM выдают соизмеримые качества предсказаний. Для этого необходимо лишь правильно подбирать гиперпараметры. Далее все зависит от целей использования модели. Если у нам важно скорость выдачи предсказаний, и есть достаточное время на обучение модели, то лучше использовать CatBoost. Если же наоборот предсказания нужны раньше по времени, то лучше использовать LightGBM, оно обучается быстрее, и засчет этого первые предсказания появтся раньше, чем в CatBoost.<br>
Для нашего набора данных лучшие значения показал LightGBM. Он показал высокое качество и быстрое обучение.

 ## 4. Общий вывод

В данной работе мы изучали работу алгоритмов градиентного бустинга, таких как CatBoost и LightGBM. С их использованием, нам необходимо было построить модель для определения стоимости автомобиля.<br>
Изначально мы подготовили данные, заполнили пропущенные значения. Далее обучили три разных модели. Для каждой по своему подготовили категориальные признаки, а также провели стандратизацию количественных значений. Измерили качество работы, использовав для этого метрику RMSE. Также измерили время обучения и предсказания В итоге проанализировав все полуенные характеристики, выбрали лучшую модель для этого набра данных. Ей оказалась LightGBM.